In [11]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.effects import compress_dynamic_range
import re
import __future__
import os
import io
import argparse
from multiprocessing import Pool
import cyrtranslit
import glob
import logging as log
import MailDownloader
from google_classifier import Classifier

In [4]:
# Vars:
SILENCE_AT_BEGINNING_AND_END_MS = 3000
APPLAUSE = "Applause"

In [12]:

output_path = "C:\\Users\\Eitan\\Music\\dev\\Handclap detection"
preferred_codec = 'wav'

classification_results = {}

In [12]:
def create_folder():
    #TODO: change when going back to prod
    folder = "test"
    path = os.path.join(output_path, folder)

    if os.path.exists(path):
        print('Folder exists')
        
    else:
        os.makedirs(path)
        print('Created folder ' + folder)

    os.chdir(path)


def parse_email(mail):
    try:
        url = re.match("(https?://[^\s]+)", mail).group(0)
        url = re.sub("&list=.*", "", url)
        return url
    except:
        pass


def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)


def print_exceptions(exceptions):
    if len(exceptions['WAV']) > 0:
        log.error("### WAV Exceptions ###")
        for i in exceptions['WAV']:
            log.error(i)

    if len(exceptions['Video']) > 0:
        log.error("### Video Exceptions ###")
        for i in exceptions['Video']:
            log.error(i)


def print_found_urls(urls):
    print("1. Found the following URIs, {0} in total:".format(len(urls)))
    for i in urls:
        print(i)

        
def normalize_all_filenames():
    files = get_file_list()
    print("3. Normalizing all file names")
    for file in files:
        normalize_filename(file)


def normalize_filename(file):
    if not file.endswith('.wav'):
        return

    print("3.1 Normalizing {0}".format(file))
    #fix cyrillic characters
    new_name = re.sub('\W+', ' ', cyrtranslit.to_latin(file, "ru"))
    new_name = re.sub(' wav', ".wav", new_name)
    os.replace(file, new_name)
    print("3.1 Replaced. Old file name: {0}, New file name: {1}".format(file, new_name))


def compress_all_files():
    files = get_file_list()
    print(f"4. Compressing all files")
    with Pool() as pool:
        pool.map(compress_file, files)
    print("4. Finished compressing")


def compress_file(file):
    if not file.endswith('.wav'):
        pass
    else:
        print(f'4.1 Compressing: -- {file}')
        raw_sound = AudioSegment.from_file(file)
        compressed_sound = compress_dynamic_range(raw_sound, -20, 6)
        new_file = re.sub("\.wav", "_C.wav", file)
        compressed_sound.export(new_file , format='wav')

        print("Deleting " + file)
        os.remove(file)

        print(f"4.1 Compressed {new_file}")


def normalize_all_audio_files():
    files = get_file_list()
    print("5. Normalizing files...")
    with Pool() as pool:
        pool.map(normalize_audio_file, files)
    print("5. Finished normalizing")


def normalize_audio_file(file):
    if not file.endswith('.wav'):
        pass

    else:
        print(f"5.1 Normalizing {file}")
        raw_sound = AudioSegment.from_file(file)
        normalized_file = normalize(raw_sound, -1)
        new_file = re.sub("\.wav", "N.wav", file)
        normalized_file.export(new_file, format='wav')

#BUG: not deleting N object
        print("Deleting " + file)
        os.remove(file)   
        print(f"5.1 Normalized {new_file}")     


def add_ms_of_silence_to_all_files():
    files = get_file_list()
    print("6. Add silence to files")
    with Pool() as pool:
        pool.map(add_silence_to_file, files)
    print("6. Finished adding silence ")


def add_silence_to_file(file):
    print(f"6.1 Adding silence to file {file}")
    raw_sound = AudioSegment.from_file(file)
    sample = raw_sound[:SILENCE_AT_BEGINNING_AND_END_MS]
    sample = sample - 70
    output = sample + raw_sound + sample
    new_file = re.sub("\.wav", "S.wav", file)
    output.export(new_file, format='wav')

    print("Deleting " + file)
    os.remove(file)

    print(f"6.1 Added silence to {new_file}")


def read_email():
    email_content = []
    mail = """
        Peter Heidrich - Happy Birthday Variations (excerpts)
    https://www.youtube.com/watch?v=BmA0bbZSDN4 from 1.07 cut applause in the end please
    Gidon Kremer, violin. Vadim Repin, Misha Maisky etc (10-12)
    Tchaikovsky - Waltz from Sleeping Beauty ----- Чайковский - Вальс из балета "Спящая красавица"
    https://www.youtube.com/watch?v=Gb8Xtix7jc8 4.47
    Handel, Birthday Ode for Queen Anne - 5. DUET & CHORUS, "Let rolling streams" (score)
    https://www.youtube.com/watch?v=DsXBTMabMaA 2.46
    Kathleen Battle & Wynton Marsalis, 'Eternal Source of Light Divine' (G. F. Haendel)          (Ode for the birthday of Queen Anne in 1713).
    https://www.youtube.com/watch?v=2MuCCbg0k_0 3.08         и сопрано, и трубач- негры
    Jean Sibelius: Cinq morceaux op. 75 No. 5: "Granen" / "the Spruce" (1919) Ralf Gothóni 1995
    https://www.youtube.com/watch?v=VbUHH8NiquM 3.25    Ель из цикла Деревья
    """
    try:
        for line in mail.split('\r\n'):
            print(line)
            email_content.append(line)
    except KeyboardInterrupt as e:
        pass

    return email_content


def parse_all_email_lines(email_content):
    urls = []

    for line in email_content:
        urls.append(parse_email(line))

    return urls


def find_urls_in_email(email_content):
    urls = parse_all_email_lines(email_content)
    urls = list(filter(None, urls))
    return urls


def get_file_list():
    return glob.glob("*.wav")
    

def convert_to_mp3():
    files = get_file_list()
    for file in files:
        audio = AudioSegment.from_wav(file)
        filename = file.replace("wav","mp3")
        audio.export(filename,format='mp3')
    
        print("Deleting " + file)
        os.remove(file)   


def configure_log():
    log.basicConfig(
    level=log.INFO,
    format='%(asctime)s,%(funcName)s,%(levelname)s,%(message)s',
    handlers=[
        log.FileHandler("debug.log"),
        log.StreamHandler()
        ]
    )


In [21]:
email_content = read_email()
urls = find_urls_in_email(email_content)
print_found_urls(urls)

1. Found the following URIs, 0 in total:


In [17]:
mail = read_email()
print(mail)

['\n        Peter Heidrich - Happy Birthday Variations (excerpts)\n    https://www.youtube.com/watch?v=BmA0bbZSDN4 from 1.07 cut applause in the end please\n    Gidon Kremer, violin. Vadim Repin, Misha Maisky etc (10-12)\n    Tchaikovsky - Waltz from Sleeping Beauty ----- Чайковский - Вальс из балета "Спящая красавица"\n    https://www.youtube.com/watch?v=Gb8Xtix7jc8 4.47\n    Handel, Birthday Ode for Queen Anne - 5. DUET & CHORUS, "Let rolling streams" (score)\n    https://www.youtube.com/watch?v=DsXBTMabMaA 2.46\n    Kathleen Battle & Wynton Marsalis, \'Eternal Source of Light Divine\' (G. F. Haendel)          (Ode for the birthday of Queen Anne in 1713).\n    https://www.youtube.com/watch?v=2MuCCbg0k_0 3.08         и сопрано, и трубач- негры\n    Jean Sibelius: Cinq morceaux op. 75 No. 5: "Granen" / "the Spruce" (1919) Ralf Gothóni 1995\n    https://www.youtube.com/watch?v=VbUHH8NiquM 3.25    Ель из цикла Деревья\n    ']
